In [ ]:
#!pip install einops accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
from PIL import Image
import requests
import torch

In [ ]:
processor = AutoProcessor.from_pretrained(
    'allenai/Molmo-72B-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    'allenai/Molmo-72B-0924',
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

In [ ]:
urls = [
 'https://samplebook.photos/img/4786a.jpg',
 'https://samplebook.photos/img/284v.jpg',
 'https://samplebook.photos/img/3193c.jpg',
 'https://samplebook.photos/img/948d.jpg',
 'https://samplebook.photos/img/3191i.jpg',
 'https://samplebook.photos/img/4788oo.jpg',
 'https://samplebook.photos/img/2086d.jpg',
 'https://samplebook.photos/img/4781r.jpg',
 'https://samplebook.photos/img/5521n.jpg',
 'https://samplebook.photos/img/2073cc.jpg',
 'https://samplebook.photos/img/2085b.jpg',
 'https://samplebook.photos/img/4782c.jpg',
 'https://samplebook.photos/img/4783v.jpg',
 'https://samplebook.photos/img/4912gg.jpg',
 'https://samplebook.photos/img/304h.jpg',
 'https://samplebook.photos/img/2078i.jpg',
 'https://samplebook.photos/img/2077q.jpg',
 'https://samplebook.photos/img/3192h.jpg',
 'https://samplebook.photos/img/904x.jpg',
 'https://samplebook.photos/img/4792b.jpg',
 'https://samplebook.photos/img/5301p.jpg',
 'https://samplebook.photos/img/5207j.jpg',
 'https://samplebook.photos/img/1007p.jpg',
 'https://samplebook.photos/img/17l.jpg',
 'https://samplebook.photos/img/286t.jpg',
 'https://samplebook.photos/img/287g.jpg',
 'https://samplebook.photos/img/3189j.jpg',
 'https://samplebook.photos/img/4911d.jpg',
 'https://samplebook.photos/img/4785a.jpg',
 'https://samplebook.photos/img/4793a.jpg'
]

In [ ]:
prompt = 'There are two objects in this image. One is a color checker, and the other is a photographic sample book, open to a page containing a photograph. Give a detailed description of the photograph.'

In [ ]:
def get_response(url):
    
    inputs = processor.process(
        images=[Image.open(requests.get(url, stream=True).raw)],
        text=prompt
    )

    # move inputs to the correct device and make a batch of size 1
    inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}
    
    # generate output; stop generation when <|endoftext|> is generated
    output = model.generate_from_batch(
        inputs,
        GenerationConfig(max_new_tokens=1024, stop_strings="<|endoftext|>"),
        tokenizer=processor.tokenizer
    )
    
    # only get generated tokens; decode them to text
    generated_tokens = output[0,inputs['input_ids'].size(1):]
    generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_text

In [ ]:
responses_molmo = {}

for url in urls:
    response = get_response(url)
    print(url,"\n",response,"\n\n")
    responses_molmo[url] = {'allenai/Molmo-72B-0924':response}

In [ ]:
import pickle

In [ ]:
with open("responses_molmo.pkl", "wb") as f:
    pickle.dump(responses_molmo, f)